In [1]:
# !pip install llama-index llama-index-core llama-parse openai llama_index.embeddings.huggingface -q
# !pip install llama-index-llms-anthropic -q

In [2]:
  # llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv
load_dotenv()

# API access to llama-cloud
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-7K1IbMcLbyb8TDvMsIx3Brr7mD4K8ZnLaFMjbEq8S1uONYZp"
# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_APIKEY')
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

In [16]:
from llama_parse import LlamaParse

location = 'IndustrySource/Misc/32592 Explosives Manufacturing in the US Industry Report.pdf'
documents = LlamaParse(result_type="markdown").load_data(location)

INFO:httpx:HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
Started parsing the file under job_id fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b9f "HTTP/1.1 200 OK"
HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fc3fe1a8-cfdc-408e-be2a-25e6d9ac0b

'IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report'

In [17]:
INDEX_NAME = location.replace('/', '_').replace(' ', '_').split('.')[0]
INDEX_NAME

'IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report'

In [18]:
from llama_index.core.node_parser import SentenceSplitter

######## SentenceSplitter ########
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

######## Vector Index ########
from llama_index.core import VectorStoreIndex

embed_model = "local:BAAI/bge-small-en-v1.5"
vector_index = VectorStoreIndex(nodes, embed_model = embed_model)

######## GPT-4o to Chat ########
from llama_index.llms.openai import OpenAI

llm_gpt4o = OpenAI(model="gpt-4o-mini", api_key = OPENAI_API_KEY)
query_engine_gpt4o = vector_index.as_query_engine(similarity_top_k=3, llm=llm_gpt4o)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


Batches: 100%|██████████| 1/1 [00:00<00:00, 23.50it/s]
/home/yakov/anaconda3/envs/probe/lib/python3.11/site-packages/pydantic/main.py:1059: PydanticDeprecatedSince20: The `__fields__` attribute is deprecated, use `model_fields` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn(


In [19]:
query1 = "What's driving current industry performance?"
resp = query_engine_gpt4o.query(query1)
print("GPT-4o-mini:")
print(str(resp))

Batches: 100%|██████████| 1/1 [00:00<00:00, 43.96it/s]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
GPT-4o-mini:
Current industry performance is being driven by several key factors. Rising interest rates have limited demand from the construction sector, which is a significant source of revenue for the industry, particularly in residential construction. Additionally, the Russian invasion of Ukraine has disrupted supply chains, particularly for crude oil products, leading to increased costs for manufacturers. The trade market is also facing severe contractions, with a strong US dollar making imports more attractive than domestic products. These elements combined are impacting the overall performance of the industry.


In [20]:
str(resp)

'Current industry performance is being driven by several key factors. Rising interest rates have limited demand from the construction sector, which is a significant source of revenue for the industry, particularly in residential construction. Additionally, the Russian invasion of Ukraine has disrupted supply chains, particularly for crude oil products, leading to increased costs for manufacturers. The trade market is also facing severe contractions, with a strong US dollar making imports more attractive than domestic products. These elements combined are impacting the overall performance of the industry.'

In [21]:
print(nodes[12].text)

# IBISWorld | Explosives Manufacturing in the US

# Mar 2024

# Performance

Track historical, current and forward-looking trends in revenue, profit and other performance indicators that make or break an industry.


In [22]:
query1 = "Tell me about Oil States International Inc. to acquire GEODynamics Inc."
resp = query_engine_gpt4o.query(query1)
print("GPT-4o-mini:")
str(resp)

Batches: 100%|██████████| 1/1 [00:00<00:00, 44.70it/s]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
GPT-4o-mini:


"Oil States International Inc. completed the acquisition of GEODynamics Inc. on January 12, 2018, in a deal valued at $615 million. This acquisition allowed Oil States to gain full ownership of GEODynamics, with plans to leverage its services to expand its influence and customer base in the technology sector. Additionally, the acquisition aimed to enhance Oil States' own offerings in the oil industry by utilizing GEODynamics' expertise in oil and gas solutions. Despite the acquisition, GEODynamics continues to operate as a separate entity under its own name."

In [23]:
#####################################################################################################################################################

In [24]:
# WEAVIATE Vector Database -RAG

In [25]:
%pip install llama-index-vector-stores-weaviate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [26]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from IPython.display import Markdown, display

In [27]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [28]:
import weaviate
# cloud
cluster_url = "https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud"
api_key = "7ZfUCibywHnzM0WKMPx7YevuN79nUtS4KJgT"

client = weaviate.connect_to_wcs(
    cluster_url=cluster_url,
    auth_credentials=weaviate.auth.AuthApiKey(api_key),
)

INFO:httpx:HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"


In [29]:
from llama_index.core import StorageContext

# If you want to load the index later, be sure to give it a name!
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name=INDEX_NAME
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
vector_index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

INFO:httpx:HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 404 Not Found"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 404 Not Found"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"
HTTP Request: POST https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.

In [30]:
# set Logging to DEBUG for more detailed outputs
query_engine = vector_index.as_query_engine()
response = query_engine.query("Tell me about Oil States International Inc. to acquire GEODynamics Inc.")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
HTTP 

In [31]:
display(Markdown(f"{response}"))

Oil States International Inc. completed the acquisition of GEODynamics Inc. on January 12, 2018, in a $615 million deal. Following the acquisition, Oil States International Inc. now has full ownership of GEODynamics Inc. The purpose of this acquisition is to leverage GEODynamics Inc.'s expertise and services to expand Oil States International Inc.'s presence in the technology sector and enhance its own services within the oil industry. Despite the acquisition, GEODynamics Inc. will continue to operate as a separate entity under its existing name.

In [32]:
#########################################################################################################################################################

In [33]:
# weaviate vector database & llamaparse Integrated

In [34]:
from llama_index.core.node_parser import SentenceSplitter

######## SentenceSplitter ########
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [35]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name=INDEX_NAME
)

INFO:httpx:HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"
HTTP Request: GET https://a0dlgmcaspopjrn2mtx4ha.c0.us-east1.gcp.weaviate.cloud/v1/schema/IndustrySource_Misc_32592_Explosives_Manufacturing_in_the_US_Industry_Report "HTTP/1.1 200 OK"


In [36]:
######## Vector Index ########
from llama_index.core import VectorStoreIndex
vector_index = VectorStoreIndex(nodes, vector_store = vector_store)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [37]:
######## GPT-4o to Chat ########
from llama_index.llms.openai import OpenAI

llm_gpt4o = OpenAI(model="gpt-4o-mini", api_key = OPENAI_API_KEY)
query_engine_gpt4o = vector_index.as_query_engine(similarity_top_k=3, llm=llm_gpt4o)

In [38]:
query1 = "What's driving current industry performance?"
resp = query_engine_gpt4o.query(query1)
print("GPT-4o-mini:")
str(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
GPT-4o-mini:


"Current industry performance is influenced by several key factors. Rising interest rates have limited demand from the construction sector, which is a significant source of revenue for the industry. This has particularly affected residential construction, which experienced strong growth during the pandemic. \n\nAdditionally, the Russian invasion of Ukraine has disrupted supply chains, particularly for crude oil products, which are essential inputs for explosives manufacturing. The resulting increase in crude oil prices has impacted profit margins, as manufacturers face higher costs.\n\nTrade markets are also experiencing severe contractions, with a decline in export revenues and a strong US dollar making imports more attractive than domestic products. This has further strained the industry's performance. \n\nOn a more positive note, demand from coal mining and other mining sectors, along with the value of construction and the price of nonferrous metals, are contributing positively to t

In [39]:
query1 = "Tell me about Oil States International Inc. to acquire GEODynamics Inc."
resp = query_engine_gpt4o.query(query1)
print("GPT-4o-mini:")
str(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
GPT-4o-mini:


"Oil States International Inc. completed the acquisition of GEODynamics Inc. on January 12, 2018, in a deal valued at $615 million. This acquisition allowed Oil States to gain full ownership of GEODynamics, with plans to leverage its expertise and services to enhance its influence and customer base within the technology sector. Additionally, the acquisition aimed to improve Oil States' own offerings in the oil industry. Despite the acquisition, GEODynamics continues to operate as a separate entity under its own name."